In [14]:
!pip install selenium
!apt-get -q update
!apt install -yq chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
#specify driver path
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

import pandas as pd
driver = webdriver.Chrome('chromedriver', options = chrome_options)

'apt-get' 不是内部或外部命令，也不是可运行的程序
或批处理文件。
'apt' 不是内部或外部命令，也不是可运行的程序
或批处理文件。
'cp' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


## Get jobs and job descriptions

In [2]:
def find_jobs(cat, titles, companies, categories, links, reviews):
    for i in range(0,15):
    
        job_card = driver.find_elements_by_xpath('//div[contains(@class,"clickcard")]')

        for job in job_card:

            categories.append(cat)

        #.  not all companies have review
            try:
                review = job.find_element_by_xpath('.//span[@class="ratingsContent"]').text
            except:
                review = "None"
            reviews.append(review)


            try:
                title  = job.find_element_by_xpath('.//h2[@class="title"]//a').text
            except:
                title = job.find_element_by_xpath('.//h2[@class="title"]//a').get_attribute(name="title")
            titles.append(title)
            links.append(job.find_element_by_xpath('.//h2[@class="title"]//a').get_attribute(name="href"))
            companies.append(job.find_element_by_xpath('.//span[@class="company"]').text)


        try:
            next_page = driver.find_element_by_xpath('//a[@aria-label={}]//span[@class="pn"]'.format(i+2))
            next_page.click()
            
        except:
            next_page = driver.find_element_by_xpath('//a[@aria-label="Next"]//span[@class="np"]')
            next_page.click()
        #except:
            #next_page = driver.find_element_by_xpath('//a[.//span[contains(text(),"Next")]]')
            #next_page.click()


        print("Page: {}".format(str(i+1)))
    
    return titles, companies, categories, links, review

In [4]:
titles = []
companies = []
categories = []
links = []
reviews = []
jobs = ["data analyst", "data scientist", "business analyst", "financial analyst"]

for j in jobs:
    for state in ["TX", "CA", "NY"]:
        driver.get('https://www.indeed.com/advanced_search?q=&l=' + state)
        print(j, "in", state)
        driver.implicitly_wait(3) 

        #search data
        search_job = driver.find_element_by_xpath('//input[@id="as_and"]')
        search_job.send_keys([j])

        #set display limit of 20 results per page
        display_limit = driver.find_element_by_xpath('//select[@id="limit"]//option[@value="20"]')
        display_limit.click()

        #sort by date
        sort_option = driver.find_element_by_xpath('//select[@id="sort"]//option[@value="date"]')
        sort_option.click()
        search_button = driver.find_element_by_xpath('//*[@id="fj"]')
        search_button.click()

        window_after = driver.window_handles[-1]
        driver.switch_to.window(window_after)

        titles, companies, categories, links, review = find_jobs(j, titles, companies, categories, links, reviews)

data analyst in TX
Page: 1
Page: 2
Page: 3
Page: 4
Page: 5
Page: 6
Page: 7
Page: 8
Page: 9
Page: 10
Page: 11
Page: 12
Page: 13
Page: 14
Page: 15
data analyst in CA
Page: 1
Page: 2
Page: 3
Page: 4
Page: 5
Page: 6
Page: 7
Page: 8
Page: 9
Page: 10
Page: 11
Page: 12
Page: 13
Page: 14
Page: 15
data analyst in NY
Page: 1
Page: 2
Page: 3
Page: 4
Page: 5
Page: 6
Page: 7
Page: 8
Page: 9
Page: 10
Page: 11
Page: 12
Page: 13
Page: 14
Page: 15
data scientist in TX
Page: 1
Page: 2
Page: 3
Page: 4
Page: 5
Page: 6
Page: 7
Page: 8
Page: 9
Page: 10
Page: 11
Page: 12
Page: 13
Page: 14
Page: 15
data scientist in CA
Page: 1
Page: 2
Page: 3
Page: 4
Page: 5
Page: 6
Page: 7
Page: 8
Page: 9
Page: 10
Page: 11
Page: 12
Page: 13
Page: 14
Page: 15
data scientist in NY
Page: 1
Page: 2
Page: 3
Page: 4
Page: 5
Page: 6
Page: 7
Page: 8
Page: 9
Page: 10
Page: 11
Page: 12
Page: 13
Page: 14
Page: 15
business analyst in TX
Page: 1
Page: 2
Page: 3
Page: 4
Page: 5
Page: 6
Page: 7
Page: 8
Page: 9
Page: 10
Page: 11
Page: 12
Pa

In [10]:
i = 0
descriptions = []
for link in links:
    driver.get(link)
    try:
        jd = driver.find_element_by_xpath('//*[@id="jobDescriptionText"]').text
        jd = " ".join(jd.split("\n"))
    except:
        jd = "None"
        print(i)
    descriptions.append(jd)
    i += 1

In [11]:
df = pd.DataFrame()
df['Title'] = titles
df['Company'] = companies
df['Category'] = categories
df['Link'] = links
df['Review'] = reviews
df['Description'] = descriptions

In [12]:
df.to_excel("jobs.xlsx")

In [3]:
df = pd.read_excel("jobs.xlsx")
df

,Unnamed: 0,Title,Company,Location,Link,Review,descriprion
0,0,Teacher Assistance,School of science and technology Excellence,"Houston, TX",https://www.indeed.com/company/School-Of-Scien...,None,Note: SST Contribution to Health Benefits is $...
1,1,Drone Operator,"ProCam Pilots, LLC","Houston, TX",https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,None,ProCam Pilots is now looking for certified pil...
2,2,Adminstrative Assistant,Leaf Technologies Inc,"Houston, TX",https://www.indeed.com/company/Leaf-Technologi...,None,"We are looking for a energetic, self motivated..."
3,3,Substitute Teacher,School of Science and Technology - Spring,"Houston, TX",https://www.indeed.com/company/School-of-Scien...,None,"School of Science and Technology (SST) is an ""..."
4,4,Administrative Assistant,Renee Kennedy,"Houston, TX",https://www.indeed.com/company/Renee-Kennedy/j...,None,Looking for someone to help me with some of th...
...,...,...,...,...,...,...,...
5595,5595,Salesforce Technical Lead - Permanent or contr...,Platform 6 Ltd,"Silicon Valley, CA",https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,None,This leading and growing Salesforce organisati...
5596,5596,Data Center Technician II,Milestone Technologies,"Silicon Valley, CA",https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,None,"Company Overview Milestone Technologies, Inc. ..."
5597,5597,Systems Engineer - Data Loss Prevention,The Mom Project,"Silicon Valley, CA",https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,None,Our Customer is an American business and finan...
5598,5598,Data Center Visitor Management Security Officer,Inter-Con Security Systems,"Silicon Valley, CA",https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,3.2,"Company Overview: Founded in 1973, Inter-Con S..."
